<h1><center>Final Assignement</center></h1>

Student number: 200420653

<a class="anchor" id="0"></a>
## Table of content
1. [Introduction and objectives](#1)<br>
    1.1 [Introduction](#1.1)<br>

2. [Data](#2)<br>
    2.1 [Data sources](#2.1)<br>
    2.2 [Data cleaning](#2.2)<br>
    2.3 [Data preparation](#2.3)<br>

4. [Summary](#4)<br>

**Doings:** 
<br>

Code
### - zeigen, dass Vorgehensweise dem 7 Schritte Ansatz von Chollet entspricht
- check if after first transformations a reset_index(true) is needed
- plotting the data to get some insights
- exploratory data analysis like here: https://medium.com/@trilok_/the-m5-competition-forecasting-in-store-sales-39c2e6baa7dd
- show picture of formula for RMSSE from directory ../res/misc/Formula.png
- Feature Selection: Reduce the dimensionality of data by selecting only the most important features - may involve domain knowledge or techniques like PCA or feature importance scoring.
- merge df with weekdays
- only read in data in dataframes that is needed. In case something not needed, delete it
-create classes; use special skills like the super class or an inheritance from another class; ask gpt which special skills could make sense
- If I reduce the size of the df and hash the values to integers the question remains if the models predictions would have been better without that conversion + check if float32 conversion decreases accuracy of models
- Use scalene to measure the performance of the code
- clarify how the 7 step approach from chollet book is used
- dataframe: hash id's to decrease size of df drastically. Could speed up all computations
- maybe normalization makes sense but beware of fact that only reading operations are done on the data
- it seems like I have use tensorflow, so keep in mind that it is used somewhere. Maybe just keras works

Report
- correct citation style with ACM and Mendeley

Information
- sales_amount values are originally d_1, d_2 and so on and either int16 or int8, so should not be a float value

In [1]:
#Import data handling libraries
import pandas as pd
import numpy as np

In [2]:
# Set variables
NUM_ITEMS = 30490

In [3]:
#Specify directories
# code_env = 'kaggle'
code_env = 'local'
# code_env = 'aws'


if code_env=='local':
    ###local###
    #get parent folder of current directory
    parent_dir = '/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction'

    #Directory resources
    res_dir = parent_dir + '/res/'
    src_dir = parent_dir + '/src/'
    prc_dir = src_dir + 'processed_data/' # Processed data directory with pickled dataframes

if code_env=='kaggle':
    ###On Kaggle###
    res_dir = '/kaggle/input/m5-forecasting-accuracy/'
    prc_dir = '/kaggle/input/processed-data/'

if code_env=='aws':
    parent_dir = '/home/ubuntu/projects/Kaggle_M5PointPrediction'
    res_dir = parent_dir + '/res/'
    src_dir = parent_dir + '/src/'

In [4]:
# Import the provided csv files
df_cal = pd.read_csv(res_dir + 'calendar.csv')
df_prices = pd.read_csv(res_dir + 'sell_prices.csv')
df_train_eval = pd.read_csv(res_dir + 'sales_train_evaluation.csv')
#df_train_val = pd.read_csv(res_dir + 'sales_train_validation.csv')
#df_sample_subm = pd.read_csv(res_dir + 'sample_submission.csv')

In [5]:
#Inspect dataframes
# print('\nCalendar dataframe: ')
# print(df_cal.head())
# print('\nPrices dataframe: ')
# print(df_prices.head())
# print('\nTrain evaluation dataframe: ')
# print(df_train_eval.head())


In [6]:
#Helper function to reduce memory usage of dataframes
def reduce_df_mem_usage(df, df_name):
    """ 
    Helper function to iterate all columns of given dataframe and check and set for smallest dtype to reduce memory usage
    Taken and adapted from the widely used function which is available for instance here: 
    https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage/notebook
    There is a flaw in the integer section in the publicly available function which is that it possibly introduces rounding errors when converting to a smaller dtype
    """

    #Print original memory usage
    print('Dataframe ' + df_name + ' is being processed...')
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    #Iterate through each column
    for col in df.columns:
        #Get dtype
        col_type = df[col].dtype
        
        #try except throw block to try if the following code works, otherwise skip this loop
        try:

            #If type is not an object, therefore numerical, get biggest and smallest values
            if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                
                #If type is int
                if str(col_type)[:3] == 'int':
                    #If min value is greater than min value of given dtype and max value is smaller than max value of given dtype -> adjust dtype
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                #Downcasting of float values leads to rounding errors -> as precision is crucial for float values, no downcasting is performed

            #If none of the above, then assumption that  finite set of possible values -> convert to category which is internally stored as int but when queried returns the string
            else:
                df[col] = df[col].astype('category')
        
        except:
            pass

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    print('---------------------------------------------------\n')

    return df

In [7]:
#Read in all data used later on
df_prices = reduce_df_mem_usage(df_prices, 'df_prices')
df_cal = reduce_df_mem_usage(df_cal, 'df_cal')
df_train_eval = reduce_df_mem_usage(df_train_eval, 'df_train_eval')

#delete dataframe from memory or just don't read in
#del df_sample_subm
#get name of dataframe

Dataframe df_prices is being processed...
Memory usage of dataframe is 208.77 MB
Memory usage of dataframe after optimization is: 84.90 MB
Decreased by 59.3%
---------------------------------------------------

Dataframe df_cal is being processed...
Memory usage of dataframe is 0.21 MB
Memory usage of dataframe after optimization is: 0.19 MB
Decreased by 8.7%
---------------------------------------------------

Dataframe df_train_eval is being processed...
Memory usage of dataframe is 452.91 MB
Memory usage of dataframe after optimization is: 96.30 MB
Decreased by 78.7%
---------------------------------------------------



In [8]:
df_train_eval

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


The final submission dataframe should also contain the validation data. The df_train_eval already contains this information so it will be extracted and transformed into the suitable output format. It will then be written out to a pkl file and read in and concatenated with the predictions from the model in the next notebook

In [9]:
# get column id and all columns from d_1914 to d_1941
df_val = df_train_eval.iloc[:, [0] + list(range(-28, 0))]
# rename day columns to F_1 to F_28
# Generate the new column names for columns 1 to 29
new_column_names = {df_val.columns[i]: 'F' + str(i) for i in range(1, 29)}

# Rename the columns in the DataFrame
df_val.rename(columns=new_column_names, inplace=True)

# Replace string evaluation with validation in the id column
df_val['id'] = df_val['id'].str.replace('evaluation', 'validation')

# write out
df_val.to_pickle(src_dir + 'processed_data/' + 'df_1.pkl')

# delete
del df_val

/var/folders/w1/_96k6h412s5d15f14hdtbbyc0000gn/T/ipykernel_6175/3692366094.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val.rename(columns=new_column_names, inplace=True)
/var/folders/w1/_96k6h412s5d15f14hdtbbyc0000gn/T/ipykernel_6175/3692366094.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['id'] = df_val['id'].str.replace('evaluation', 'validation')


In the following I will create 3 dataframes that will be stored in pickle format so that unused dataframes can be deleted from memory and the information is clearly separated.
The dataframes are:
- 1. df_conv: base grid with the main train information that contains the conversion of the sales data from wide to long format and merge of the sales price
- 2. calendar: contains calendar data and some generated date features
- 3. XXX: Feature engineering

First convert from wide to long format of the training dataframe. Right now all sales days are in columns. We want them in rows for processing in ML / DL models.

In [10]:
# Convert from wide to long format
df_conv = pd.melt(df_train_eval,
                    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                    var_name='d', 
                    value_name='sales_amount')

# delete df_train_eval from memory because no longer needed
del df_train_eval 

# Reduce memory usage of df_conv
df_conv = reduce_df_mem_usage(df_conv, 'df_conv')

Dataframe df_conv is being processed...
Memory usage of dataframe is 1017.24 MB
Memory usage of dataframe after optimization is: 678.68 MB
Decreased by 33.3%
---------------------------------------------------



In [11]:
# create a dataframe for the next 28 days such that a slice of all 30490 items for day 1 will be copied, the columns remain the same and only for day a new value with d_1942 - d_1969 will be generated with a NaN value for sales_amount
df_future_base = df_conv[df_conv['d']=='d_1'].copy()
# set sales_amount to NaN
df_future_base['sales_amount'] = np.nan
#create an empty dataframe that will hold the future 28 day values
df_future = pd.DataFrame()
#loop 28 times to concat 28 days
for i in range(28):
    df_future_base['d'] = 'd_' + str(1942 + i)
    df_future = pd.concat([df_future, df_future_base.copy()], axis=0)

In [12]:
# Now concat the future 28 days to the original dataframe
df_conv = pd.concat([df_conv, df_future], axis=0)
df_conv.reset_index(drop=True, inplace=True)

Now we have the issue, that 0 values in the sales_amount column could mean that there were no sales, it could also mean that the sales start of the product was later and the product was just not available.
The sales start date can be infered from the price df. If there is no sales week for a product then that most likely means it wasn't available for sale.
I will first try to delete all rows from df_conv where there were no weeks available for from the price df. This will be done by adding the week information from prices through a left join. Then the week information is available to make a right join on the prices table and thereby filter out all rows where there was no sales week available (the product didn't exist yet).

The reason why I didn't convert float types is that they loose precision. That is absolutely crucial for sales prediction. That is why now the dataframe is too large for the merge operation in the following. Even 30GB RAM machines run out of memory on this computation as it require much more memory than the original ~680MB for df_conv and ~85MB for Prices. The solution for me is using DASK which handles operations on dataframes in smaller chunks.

In [13]:
####### Cited from the 1st solution (exact citing) #######
## Merging by concat to not lose dtypes
def merge_by_concat_left(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [14]:
# Combine the converted df with the calendar df
df_conv = merge_by_concat_left(df_conv, df_cal[['d', 'wm_yr_wk']], ['d'])

In [15]:
# Check output of operation
df_conv.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales_amount,wm_yr_wk
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101


Merge with the prices df to get the sales start date for each product. Then filter out all rows where there was no price available (the product didn't exist yet).

Feature engineering for prices can be done here, and then can in the next step be merged with the df_conv dataframe and deleted from memory
- Promotions
- Average sales per month
- Explicit weighting with sales volume per store, country and so on because the final evaluation is being done with WRMSSE 
- Weight more recent sales higher than older sales


In [16]:
# Combine the converted df with the prices df
df_conv = merge_by_concat_left(df_conv, df_prices, ['store_id', 'item_id', 'wm_yr_wk'])

In [17]:
# Create a new column is_available where when column sell_price is NaN the value is 0 (indicates item was not available) and otherwiese 1 (indicates the item was available that day)
df_conv['is_available'] = np.where(df_conv['sell_price'].isna(), 0, 1).astype(np.int8)

# Also set column sales_amount to NaN when sell_price is NaN
#df_conv['sales_amount'] = np.where(df_conv['sell_price'].isna(), 0, df_conv['sales_amount'])
df_conv['sell_price'].fillna(0, inplace=True)

In [18]:
# Check output of operation
df_conv.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales_amount,wm_yr_wk,sell_price,is_available
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101,0.0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101,0.0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101,0.0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101,0.0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,11101,0.0,0


In [19]:
# Perform feature engineering
"""
#####- these columns have to be update in the next notebook based on the predictions made by the model #####
- 1 days lag #float 64
- moving average for 7 and 28 days #float 64
- is there a price reduction?
- is there a price increase?
- adjust for inflation?
- consumer sentiment
- holiday
- weather
- 
"""
def feature_engineering(df): 
    ################## lag 1 day sales amount ##############################################################################
    df_conv['sales_amount_lag_1'] = df_conv['sales_amount'].shift(NUM_ITEMS)

    # Now the first value of every entry in the sales_amount_lag_1 column is NaN, because there is no value for the first day of the time series, so we will replace these with the mode (most frequent value) of the sales_amount column
    mode_sales_amount = df_conv.groupby('id')['sales_amount'].agg(lambda x: x.mode()[0])

    # Replace the first day's sales_amount_lag_1 for each item with the mode_sales_amount value
    df_conv.loc[df_conv['d'] == 'd_1', 'sales_amount_lag_1'] = df_conv.loc[df_conv['d'] == 'd_1', 'id'].map(mode_sales_amount)
    ########################################################################################################################

    ################## moving average 7 and 28 days #########################
    df_conv['sales_amount_moving_avg_7'] = df_conv.groupby('id')['sales_amount'].transform(lambda x: x.rolling(window=7).mean())
    df_conv['sales_amount_moving_avg_7'] = df_conv['sales_amount_moving_avg_7'].fillna(method='bfill')

    df_conv['sales_amount_moving_avg_28'] = df_conv.groupby('id')['sales_amount'].transform(lambda x: x.rolling(window=28).mean())
    df_conv['sales_amount_moving_avg_28'] = df_conv['sales_amount_moving_avg_28'].fillna(method='bfill')
    #########################################################################

    ################## days consecutive zero sales and if an entry means that this is a zero sale  #########################
    # Step 1: Mark zero sales days where item is available
    df_conv['zero_sales_available'] = np.where((df_conv['sales_amount'] == 0) & (df_conv['is_available'] == 1), 1, 0).astype(np.int8)

    # Function to apply to each group
    def calculate_consecutive_zeros(group):
        # Step 2: Identify change points to reset the count for consecutive zeros
        group['block'] = (group['zero_sales_available'] == 0).cumsum().astype(np.int16)
        
        # Step 3: Count consecutive zeros within each block
        group['consecutive_zero_sales'] = group.groupby('block').cumcount()
        
        # Reset count where 'zero_sales_available' is 0, as these are not zero sales days or the item is not available
        group['consecutive_zero_sales'] = np.where(group['zero_sales_available'] == 1, group['consecutive_zero_sales'], 0).astype(np.int16)
        
        return group

    # Apply the function to each item group
    df_conv = df_conv.groupby('id', group_keys=False).apply(calculate_consecutive_zeros)

    # Drop the 'block' column because no longer needed
    del df_conv['block']

    return df
########################################################################################################################

In [ ]:
df_conv = feature_engineering(df_conv)

In [20]:
df_conv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60034810 entries, 0 to 60034809
Data columns (total 16 columns):
 #   Column                      Dtype   
---  ------                      -----   
 0   id                          category
 1   item_id                     category
 2   dept_id                     category
 3   cat_id                      category
 4   store_id                    category
 5   state_id                    category
 6   d                           object  
 7   sales_amount                float64 
 8   wm_yr_wk                    int16   
 9   sell_price                  float64 
 10  is_available                int8    
 11  sales_amount_lag_1          float64 
 12  sales_amount_moving_avg_7   float64 
 13  sales_amount_moving_avg_28  float64 
 14  zero_sales_available        int8    
 15  consecutive_zero_sales      int16   
dtypes: category(6), float64(5), int16(2), int8(2), object(1)
memory usage: 5.9+ GB


In [21]:
# df_conv[(df_conv['id']== 'HOBBIES_1_001_CA_1_evaluation') & (df_conv['is_available'] == 1)].head(50)
# df_conv.head()
# df_conv_test[df_conv_test['id']== 'HOBBIES_1_001_CA_1_evaluation'].head(50)

In [22]:
# delete df_prices from memory because no longer needed
del df_prices
del df_conv['wm_yr_wk']
df_conv.reset_index(drop=True, inplace=True)

In [23]:
# Base steps are done, now lets save to pickle file
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_2.pkl')
del df_conv

Before we move on to the merge the infomation from the calendar to the df_conv dataframe, it makes sense to perform some feature engineering based on the prices information as long as the df_conv is smaller without the calendar information.

### Create the second dataframe with the calendar information
Now lets get the calendar information and do some feature engineering for a third dataframe.

In [ ]:
# read df_conv from pickle file 
df_conv = pd.read_pickle(src_dir + 'processed_data/' + 'df_2.pkl')

In [ ]:
# Retrieve only identifying columns of df_conv to keep df small and to merge with df_conv
df_conv = df_conv[['id', 'd']]

In [ ]:
# Pick all useful columns for merge with df_conv
icols = ['date',
         'd',
         'wday',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

# Now merge the df_conv with useful features from the prices df
df_conv = merge_by_concat_left(df_conv, df_cal[icols], ['d'])

In [ ]:
# Convert the date column to datetime format
df_conv['date'] = pd.to_datetime(df_conv['date'])

# Do feature engineering on the df_conv dataframe; more features are possible
# df_conv['mday'] = df_conv['date'].dt.day.astype(np.int8)
df_conv['week'] = df_conv['date'].dt.isocalendar().week.astype(np.int8)
df_conv['month'] = df_conv['date'].dt.month.astype(np.int8)
df_conv['year'] = df_conv['date'].dt.year.astype(np.int16)
# df_conv['tm_y'] = (df_conv['tm_y'] - df_conv['tm_y'].min()).astype(np.int8)
# df_conv['tm_wm'] = df_conv['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

# df_conv['tm_dw'] = df_conv['date'].dt.dayofweek.astype(np.int8) 
# df_conv['tm_w_end'] = (df_conv['tm_dw']>=5).astype(np.int8)

df_conv['d'] = df_conv['d'].str.replace('d_', '').astype(np.int16)

In [ ]:
######################################## Convert to cyclical data of date time ########################################
# Should be moved to first script
# Convert cyclical features with sin and cos
df_conv['month_sin'] = np.sin(2 * np.pi * df_conv['month']/12).astype('float16')
df_conv['month_cos'] = np.cos(2 * np.pi * df_conv['month']/12).astype('float16')
df_conv.drop('month', axis=1, inplace=True)

df_conv['wday_sin'] = np.sin(2 * np.pi * df_conv['wday']/7).astype('float16')
df_conv['wday_cos'] = np.cos(2 * np.pi * df_conv['wday']/7).astype('float16')
df_conv.drop('wday', axis=1, inplace=True)

df_conv['week_sin'] = np.sin(2 * np.pi * df_conv['week']/53).astype('float32')
df_conv['week_cos'] = np.cos(2 * np.pi * df_conv['week']/53).astype('float32')
df_conv.drop('week', axis=1, inplace=True)

# Normalize day of month
df_conv['total_days_in_month'] = df_conv['date'].dt.days_in_month
df_conv['mday_normalized'] = ((df_conv['date'].dt.day - 1) / (df_conv['total_days_in_month'] - 1)).astype('float32')
df_conv.drop(['date', 'total_days_in_month'], axis=1, inplace=True)

# Create a continous, normalized day number
df_conv['day_continuous_normalized'] = (df_conv['d'] / 1969).astype('float32')  # 1969 train, val and test days in total

# Normalize linear data like year
min_year = df_conv['year'].min()
max_year = df_conv['year'].max()
df_conv['year_normalized'] = ((df_conv['year'] - min_year) / (max_year - min_year)).astype('float16')
df_conv.drop('year', axis=1, inplace=True)
########################################################################################################################

In [ ]:
# save to pickle and delete df_cal from memory because no longer needed
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_3.pkl')
del df_cal
del df_conv

In [ ]:
# Now the date column with the old format can be deleted from the df_conv dataframe
df_conv = pd.read_pickle(src_dir + 'processed_data/' + 'df_2.pkl')
del df_conv['d']
del df_conv['id']
df_conv.to_pickle(src_dir + 'processed_data/' + 'df_2.pkl')